In [1]:
import pandas as pd
import os
import re
from typing import List
import math
import numpy as np
from scipy.interpolate import BPoly
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%load_ext autoreload
%autoreload 2

params = {"font.family": "Arial", 'mathtext.default': 'regular'}      
plt.rc('font', size=8)
plt.rcParams.update(params)
plt.rcParams['axes.facecolor'] = 'white'

%matplotlib inline

In [2]:
def get_second_order_barrier_correction(concentration, temperature):
    return math.log(1/concentration)*temperature*8.314/1000/4.184

def split_filename(s):
    match = re.match(r'([^-]+-[^-]+)-(.*)', s)
    if match:
        return match.groups()
    else:
        return s, ''

def get_energy(df, structure_name):
    return df[df['structure'] == structure_name]['qh-G(T)_SPC'].values[0]

def get_reference_gibbs_energy(df, structure_name):
    return df[df['label_name'] == structure_name]['reference_gibbs'].values[0]

def get_reactant_energy(structure_name):
    if "naphh" in structure_name:
        structure_name = "naphh"
    return base_reactants_df[base_reactants_df['structure'] == structure_name]['qh-G(T)_SPC'].values[0]

def get_ligand_energy(df, ligand):
    return df[(df['group'] == ligand) & (df['structure'] == '84-lig')]['qh-G(T)_SPC'].values[0]

def get_reference_energy(df, ligand):
    return df[(df['structure'] == '00-lpdoh2') & (df['group'] == ligand)]['equalized_gibbs'].values[0]

def get_precatalyst_energy(df, ligand):
    return (get_energy(df, "99-l2pd2oh4") - 2*get_ligand_energy(ligands_df, ligand))

def equalize_reference(row, ligand):
    if row['structure'] == '00-lpdoh2':
        return row['qh-G(T)_SPC'] / 2 + 2*get_energy(base_reactants_df, 'naphboh2') + 2*get_energy(base_reactants_df, 'h2o') + get_ligand_energy(ligands_df, ligand)
    
    elif row['structure'] in ['01-rxt', '02-ts-rxt-c1', '03-c1', '04-ts-c1-c2', '05-c2',
                            '12-ts-rxt-t1', '13-t1', '14-ts-t1-t2', '15-t2', 
                            '27-ts-t2-xa',  '30-ts-t2-xb',  
                            '33-ts-rxt-ya', '34-ya', '35-ts-ya-c1', '58-c2alt',
                            '59-ts-t2-t3rh', '60-t3rh', '61-ts-t3rh-t3ob', '62-t3ob-rh',
                            ]:
        return row['qh-G(T)_SPC'] + 2*get_energy(base_reactants_df, 'h2o') + get_energy(base_reactants_df, 'naphboh2') + get_ligand_energy(ligands_df, ligand)
    
    elif row['structure'] in ['06-c2-h2o', '07-ts-c2-c3', '08-c3-boh3',
                            '16-t2-h2o', '17-ts-t2-t3', '18-t3-boh3',
                            '22-c5', '23-ts-c5-c6', '24-t5', '25-ts-t5-t6',
                            '36-ts-rxt-yb', '37-yb', '38-ts-yb-c1', '26-t6',
                            ]:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'naphboh2') + get_ligand_energy(ligands_df, ligand) + get_energy(base_reactants_df, 'h2o')
    
    elif row['structure'] in ['28-xa', '31-xb',]:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'boh3') + 2*get_energy(base_reactants_df, 'h2o') + get_ligand_energy(ligands_df, ligand) + get_energy(base_reactants_df, 'naphboh2')

    elif row['structure'] in ['29-ts-xa-t3', '32-ts-xb-t3',]:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'naphboh2') + get_energy(base_reactants_df, 'boh3') + get_ligand_energy(ligands_df, ligand) + get_energy(base_reactants_df, 'h2o')

    elif row['structure'] in ['09-c3', '10-ts-c3-c4', '11-c4',
                            '19-t3', '20-ts-t3-t4', '21-t4',]:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'boh3') + get_energy(base_reactants_df, 'naphboh2') + get_ligand_energy(ligands_df, ligand) + get_energy(base_reactants_df, 'h2o')
    
    elif row['structure'] in ['39-t2-lig', '40-ts-t2-p1', '41-p1-boh3',]:
        return row['qh-G(T)_SPC'] + 2*get_energy(base_reactants_df, 'h2o') + get_energy(base_reactants_df, 'naphboh2')
    
    elif row['structure'] in ['42-p1']:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'boh3') + 2*get_energy(base_reactants_df, 'h2o') + get_energy(base_reactants_df, 'naphboh2')

    elif row['structure'] in ['43-t3-lig', '44-ts-t3-p1', '45-p1-h2o',]:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'h2o') + get_energy(base_reactants_df, 'naphboh2') + get_energy(base_reactants_df, 'boh3')
    
    elif row['structure'] in ['60-t3ob',]:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'naphh') + 2*get_energy(base_reactants_df, 'h2o') + get_energy(base_reactants_df, 'naphboh2') + get_ligand_energy(ligands_df, ligand)
    
    elif row['structure'] in ['64-t3ob-1h2o', '65-ts-t3ob-1h2o-t4ob', '66-t4ob']:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'naphh') + get_energy(base_reactants_df, 'naphboh2') + get_ligand_energy(ligands_df, ligand) + get_energy(base_reactants_df, 'h2o')
    
    elif row['structure'] in ['67-t3ob-2h2o', '68-ts-t3ob-2h2o-t4ob', '69-t4ob-1h2o']:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'naphh') + get_energy(base_reactants_df, 'naphboh2') + get_ligand_energy(ligands_df, ligand)
    elif row['structure'] in ['70-c2dim']:
        return row['qh-G(T)_SPC']/2 + 2*get_energy(base_reactants_df, 'h2o') + get_energy(base_reactants_df, 'naphboh2') + get_ligand_energy(ligands_df, ligand) + get_energy(base_reactants_df, 'boh3')
    elif row['structure'] in ['71-c2-naphboh2', '72-ts-c2-naphboh2-hc1', '73-hc1-boh3']:
        return row['qh-G(T)_SPC'] + 2*get_energy(base_reactants_df, 'h2o') + get_ligand_energy(ligands_df, ligand)
    elif row['structure'] in ['72-hc1', '75-ts-hc1-hc2', '76-hc2', '77-ts-hc2-hc3', '78-hc3', '79-ts-hc3-pd0boh3']:
        return row['qh-G(T)_SPC'] + 2*get_energy(base_reactants_df, 'h2o') + get_ligand_energy(ligands_df, ligand) + get_energy(base_reactants_df, 'boh3')
    elif row['structure'] in ['80-lpd0boh3']:
        return row['qh-G(T)_SPC'] + 2*get_energy(base_reactants_df, 'h2o') + get_ligand_energy(ligands_df, ligand) + get_energy(base_reactants_df, 'boh3') + get_energy(base_reactants_df, 'binaph')
    elif row['structure'] in ['81-hc4', '82-ts-hc4-lpd0']:
        return row['qh-G(T)_SPC'] + 2*get_energy(base_reactants_df, 'h2o') + get_ligand_energy(ligands_df, ligand) + 2*get_energy(base_reactants_df, 'boh3')
    elif row['structure'] in ['83-lpd0']:
        return row['qh-G(T)_SPC'] + 2*get_energy(base_reactants_df, 'h2o') + get_ligand_energy(ligands_df, ligand) + 2*get_energy(base_reactants_df, 'boh3') + get_energy(base_reactants_df, 'binaph')
    else:
        print(f"Messed up {row}")



def get_main_microkinetics_transformations(type_name="main"):
    int_list = ['00-lpdoh2', '01-rxt', '03-c1', '05-c2', '09-c3', '11-c4', '13-t1', '15-t2', '19-t3', '21-t4', '99-l2pd2oh4']
    ts_list = ['02-ts-rxt-c1', '04-ts-c1-c2', '07-ts-c2-c3', '10-ts-c3-c4', '12-ts-rxt-t1', '14-ts-t1-t2', '17-ts-t2-t3', '20-ts-t3-t4', ]    
    rxt_pdt_list = ['naphboh2', 'h2o', 'boh3', 'naphh-c', 'naphh-t']
    diffusion_ts_list = ['diffusion-l2pd2oh4-lpdoh2', 'diffusion-lpdoh2-rxt', 'diffusion-c4-pdt', 'diffusion-t4-pdt']
    xform_dict = {'diffusion-l2pd2oh4-lpdoh2': ['99-l2pd2oh4', '', '00-lpdoh2', '00-lpdoh2', True],
                    'diffusion-lpdoh2-rxt': ['00-lpdoh2', 'naphboh2', '01-rxt', '', True],
                    '02-ts-rxt-c1': ['01-rxt', '', '03-c1', '', True],
                    '04-ts-c1-c2': ['03-c1', '', '05-c2', '', True],
                    '07-ts-c2-c3': ['05-c2', 'h2o', '09-c3', 'boh3', True],
                    '10-ts-c3-c4': ['09-c3', '', '11-c4', '', True],
                    'diffusion-c4-pdt': ['11-c4', '', 'naphh-c', '00-lpdoh2', True],
                    '12-ts-rxt-t1': ['01-rxt', '', '13-t1', '', True], 
                    '14-ts-t1-t2': ['13-t1', '', '15-t2', '', True],
                    '17-ts-t2-t3': ['15-t2', 'h2o', '19-t3', 'boh3', True],
                    '20-ts-t3-t4': ['19-t3', '', '21-t4', '', True],
                    'diffusion-t4-pdt': ['21-t4', '', 'naphh-t', '00-lpdoh2', True],  
    }
    if type_name == "main":
        pass
    elif type_name == "xa":
        int_list += ['28-xa']
        ts_list += ['27-ts-t2-xa', '29-ts-xa-t3']
        xform_dict['27-ts-t2-xa'] = ['15-t2', '', '28-xa', 'boh3', True]
        xform_dict['29-ts-xa-t3'] = ['28-xa', 'h2o', '19-t3', '', True]
    elif type_name == "xb":
        int_list += ['31-xb']
        ts_list += ['30-ts-t2-xb', '32-ts-xb-t3']
        xform_dict['30-ts-t2-xb'] = ['15-t2', '', '31-xb', 'boh3', True]
        xform_dict['32-ts-xb-t3'] = ['31-xb', 'h2o', '19-t3', '', True]
    elif type_name == "xaxb":
        int_list += ['28-xa', '31-xb']
        ts_list += ['27-ts-t2-xa', '29-ts-xa-t3', '30-ts-t2-xb', '32-ts-xb-t3']
        xform_dict['27-ts-t2-xa'] = ['15-t2', '', '28-xa', 'boh3', True]
        xform_dict['29-ts-xa-t3'] = ['28-xa', 'h2o', '19-t3', '', True]        
        xform_dict['30-ts-t2-xb'] = ['15-t2', '', '31-xb', 'boh3', True]
        xform_dict['32-ts-xb-t3'] = ['31-xb', 'h2o', '19-t3', '', True]
    elif type_name == "ya":
        int_list += ['34-ya']
        ts_list += ['33-ts-rxt-ya', '35-ts-ya-c1']
        xform_dict['33-ts-rxt-ya'] = ['01-rxt', '', '34-ya', '', True]
        xform_dict['35-ts-ya-c1'] = ['34-ya', '', '03-c1', '', True]
    elif type_name == "yb":
        int_list += ['37-yb']
        ts_list += ['36-ts-rxt-yb', '38-ts-yb-c1']
        xform_dict['36-ts-rxt-yb'] = ['01-rxt', 'h2o', '37-yb', '', True]
        xform_dict['38-ts-yb-c1'] = ['37-yb', '', '03-c1', 'h2o', True]
    elif type_name == "yayb":
        int_list += ['34-ya', '37-yb']
        ts_list += ['33-ts-rxt-ya', '35-ts-ya-c1', '36-ts-rxt-yb', '38-ts-yb-c1']
        xform_dict['33-ts-rxt-ya'] = ['01-rxt', '', '34-ya', '', True]
        xform_dict['35-ts-ya-c1'] = ['34-ya', '', '03-c1', '', True]
        xform_dict['36-ts-rxt-yb'] = ['01-rxt', 'h2o', '37-yb', '', True]
        xform_dict['38-ts-yb-c1'] = ['37-yb', '', '03-c1', 'h2o', True]
    elif type_name == "p1":
        int_list += ['42-p1']
        ts_list += ['40-ts-t2-p1', '44-ts-t3-p1']
        rxt_pdt_list += ['pre-catalyst-dimer', 'pre-catalyst-monomer', 'ligand']
        diffusion_ts_list += ['diffusion-pd2oh4-pdoh2', 'diffusion-pdoh2-l']
        xform_dict['diffusion-pd2oh4-pdoh2'] = ['pre-catalyst-dimer', '', 'pre-catalyst-monomer', 'pre-catalyst-monomer', True]
        xform_dict['diffusion-pdoh2-l'] = ['pre-catalyst-monomer', 'ligand', '00-lpdoh2', '', True]
        xform_dict['40-ts-t2-p1'] = ['15-t2', 'ligand', '42-p1', 'boh3', True]
        xform_dict['44-ts-t3-p1'] = ['19-t3', 'ligand', '42-p1', 'h2o', True]

    elif type_name == 'ob':
        int_list += ["60-t3rh", "62-t3ob-rh", "60-t3ob", "64-t3ob-1h2o", "66-t4ob", "67-t3ob-2h2o", "69-t4ob-1h2o"]
        ts_list += ["59-ts-t2-t3rh", "61-ts-t3rh-t3ob", "65-ts-t3ob-1h2o-t4ob", "68-ts-t3ob-2h2o-t4ob"]
        diffusion_ts_list += ["diffusion-59-60", "diffusion-60-61", "diffusion-61-64", "diffusion-66-63", "diffusion-63-pdt"]
        xform_dict["59-ts-t2-t3rh"] = ['15-t2', '', '60-t3rh', '', True]
        xform_dict["61-ts-t3rh-t3ob"] = ['60-t3rh', '', '62-t3ob-rh', '', True]
        xform_dict["65-ts-t3ob-1h2o-t4ob"] = ['64-t3ob-1h2o', '', '66-t4ob', '', True]
        xform_dict["68-ts-t3ob-2h2o-t4ob"] = ['67-t3ob-2h2o', '', '69-t4ob-1h2o', '', True]
        xform_dict["diffusion-59-60"] = ['62-t3ob-rh', '', '60-t3ob', 'naphh-t', True]
        xform_dict["diffusion-60-61"] = ['60-t3ob', 'h2o', '64-t3ob-1h2o', '', True]
        xform_dict["diffusion-61-64"] = ['64-t3ob-1h2o', 'h2o', '67-t3ob-2h2o', '', True]
        xform_dict["diffusion-66-63"] = ['69-t4ob-1h2o', '', '66-t4ob', 'h2o', True]
        xform_dict["diffusion-63-pdt"] = ['66-t4ob', '', '00-lpdoh2', 'boh3', True]
    elif type_name == 'hc':
        int_list += ["70-c2dim", "71-c2-naphboh2", "73-hc1-boh3", "72-hc1", "76-hc2", "78-hc3", "80-lpd0boh3", "81-hc4", "83-lpd0"]
        ts_list += ["72-ts-c2-naphboh2-hc1", "75-ts-hc1-hc2", "77-ts-hc2-hc3", "79-ts-hc3-pd0boh3", "82-ts-hc4-lpd0"]
        rxt_pdt_list += ['binaph']
        diffusion_ts_list += ['diffusion-05-70', 'diffusion-05-71', "diffusion-73-72", 'diffusion-78-81', 'diffusion-80-83']
        xform_dict["72-ts-c2-naphboh2-hc1"] = ["71-c2-naphboh2", '', "73-hc1-boh3", '', True]
        xform_dict["75-ts-hc1-hc2"] = ["72-hc1", '', "76-hc2", '', True]
        xform_dict["77-ts-hc2-hc3"] = ["76-hc2", '', "78-hc3", '', True]
        xform_dict["79-ts-hc3-pd0boh3"] = ["78-hc3", '', "80-lpd0boh3", 'binaph', True]
        xform_dict["82-ts-hc4-lpd0"] = ["81-hc4", '', "83-lpd0", 'binaph', True]
        xform_dict['diffusion-05-70'] = ['05-c2', '', '70-c2dim', 'boh3', True]
        xform_dict['diffusion-05-71'] = ['05-c2', 'naphboh2', '71-c2-naphboh2', '', True]
        xform_dict["diffusion-73-72"] = ['73-hc1-boh3', '', '72-hc1', 'boh3', True]
        xform_dict['diffusion-78-81'] = ['78-hc3', '', '81-hc4', 'boh3', True]
        xform_dict['diffusion-80-83'] = ['80-lpd0boh3', '', '83-lpd0', 'boh3', True]
    else:
        print("Invalid alternative mechanism")
    
    return int_list, ts_list, rxt_pdt_list, diffusion_ts_list, xform_dict


def get_left_right_barrier_reference_energies(df, xform: List[str]):
    if xform[1] == '':
        energy_l = float(get_energy(df, xform[0]))
    else:
        energy_l = float(get_energy(df, xform[0])) + float(get_energy(df, xform[1]))
    if xform[3] == '':
        energy_r = float(get_energy(df, xform[2]))
    else:
        energy_r = float(get_energy(df, xform[2])) + float(get_energy(df, xform[3]))
        
    return energy_l, energy_r


nice_main_names = {
    "00-lpdoh2": "[LPd(OH)$_2$]$_2$",
    "01-rxt": "RXT",
    "02-ts-rxt-c1": "RXT-C1",
    "03-c1": "C1",
    "04-ts-c1-c2": "C1-C2",
    "05-c2": "C2",
    "06-c2-h2o": "C2 + H$_2$O", 
    "07-ts-c2-c3": "C2-C3",
    "08-c3-boh3": "C3 + B(OH)$_3$", 
    "09-c3": "C3",
    "10-ts-c3-c4": "C3-C4",
    "11-c4": "C4",
    "12-ts-rxt-t1": "RXT-T1",
    "13-t1": "T1",
    "14-ts-t1-t2": "T1-T2",
    "15-t2": "T2",
    "16-t2-h2o": "T2 + H$_2$O", 
    "18-t3-boh3": "T3 + B(OH)$_3$", 
    "17-ts-t2-t3": "T2-T3",
    "19-t3": "T3",
    "20-ts-t3-t4": "T3-T4",
    "21-t4": "T4",
}

nice_altpdb_names = {
    "22-c5": "C5",
    "23-ts-c5-c6": "C5-C6",
    "24-t5": "T5",
    "25-ts-t5-t6": "T5-T6",
    "26-t6": "T6",
}

nice_xaxb_names = {
    "27-ts-t2-xa": "T2-XA",
    "28-xa": "XA",
    "29-ts-xa-t3": "XA-T3",
    "30-ts-t2-xb": "T2-XB",
    "31-xb": "XB",
    "32-ts-xb-t3": "XB-T3",
}

nice_yayb_names = {
    "33-ts-rxt-ya": "RXT-YA",
    "34-ya": "YA",
    "35-ts-ya-c1": "YA-C1",
    "36-ts-rxt-yb": "RXT-YB",
    "37-yb": "YB",
    "38-ts-yb-c1": "YB-C1",
}

nice_p1_names = {
    "39-t2-lig": "T2 + L",
    "40-ts-t2-p1": "T2-P1",
    "41-p1-boh3": "P1 + B(OH)$_3$",
    "42-p1": "P1",
    "43-t3-lig": "T3 + L",
    "44-ts-t3-p1": "T3-P1",
    "45-p1-h2o": "P1 + H$_2$O",
}

nice_alternative_names = {
    "46-ub-ref": "$\mu$B-Ref",
    "47-oacu-ref": "OAc\$mu$-Ref",
    "48-c-ref": "C-Ref",
    "49-t-ref": "T-Ref",
    "50-cl-ref": "CL-Ref",
    "51-tl-ref": "TL-Ref",
    "52-cs-ref": "CS-Ref",
    "53-ts-ref": "TS-Ref",
    "54-cw-ref": "CW-Ref",
    "55-tw-ref": "TW-Ref",
    "56-oh-ref": "OH-Ref",
    "57-koh-ref": "KOH-Ref",
    "58-c2alt": "C2$_{alt}$",
    "70-c2dim": "C2$_{dimer}$",
}

nice_ob_names = {
    "59-ts-t2-t3rh": "T2-T3RH",
    "60-t3rh": "T3RH",
    "61-ts-t3rh-t3ob": "T3RH-T3OB",
    "62-t3ob-rh": "T3OB + R-H",
    "60-t3ob": "T3OB",
    "64-t3ob-1h2o": "T3OB + H$_2$O",
    "65-ts-t3ob-1h2o-t4ob": "T3OB-H$_2$O-T4OB",
    "66-t4ob": "T4OB",
    "67-t3ob-2h2o": "T3OB + 2H$_2$O",
    "68-ts-t3ob-2h2o-t4ob": "T3OB-2H$_2$O-T4OB",
    "69-t4ob-1h2o": "T4OB + H$_2$O",
}

nice_hc_names = {
    "70-c2dim": "C2$_{dimer}$",
    "71-c2-naphboh2": "C2 + NaphB(OH)$_2$",
    "72-ts-c2-naphboh2-hc1": "C2 + NaphB(OH)$_2$-HC1",
    "73-hc1-boh3": "HC1 + B(OH)$_3$",
    "72-hc1": "HC1",
    "75-ts-hc1-hc2": "HC1-HC2",
    "76-hc2": "HC2",
    "77-ts-hc2-hc3": "HC2-HC3",
    "78-hc3": "HC3",
    "79-ts-hc3-pd0boh3": "HC3-LPd(0)B(OH)$_3$",
    "80-lpd0boh3": "LPd(0)B(OH)$_3$",
    "81-hc4": "HC4 + B(OH)$_3$",
    "82-ts-hc4-lpd0": "HC4-LPd(0)",
    "83-lpd0": "LPd(0) + B(OH)$_3$"
}

nice_unlig_names = {
    "00-murxt": "$\mu$-RXT",
    "01-rxt": "RXT",
    "02-ts-rxt-int1": "RXT-INT1",
    "03-int1": "INT1",
    "04-ts-int1-int2": "INT1-INT2",
    "05-int2": "INT2",
    "06-int2-h2o": "INT2 + H$_2$O", 
    "07-ts-int2-int3": "INT2-INT3",
    "08-int3-boh3": "INT3 + B(OH)$_3$", 
    "09-int3": "INT3",
    "10-ts-int3-int4": "INT3-INT4",
    "11-int4": "INT4",
}

nice_structure_names = {**nice_main_names, **nice_altpdb_names, **nice_xaxb_names, **nice_yayb_names, **nice_p1_names, **nice_alternative_names, **nice_ob_names, **nice_hc_names, **nice_unlig_names}

proper_names_dict = {'3q-adjohnphos': "AdJohnPhos",
 '3r-adbrettphos': "AdBrettPhos",
 '3j-brettphos': "BrettPhos",
 '3d-cyjohnphos': "CyJohnPhos",
 '3e-cymephos': "CyMePhos",
 '3f-davephos': "DavePhos",
 '3a-iprjohnphos': "(i-Pr)JohnPhos",
 '3p-me4tbuxphos': "Me$_4$tBuXPhos",
 '2k-pad3': "P(Ad)$_3$",
 '2f-pcpt3': "P(Cpt)$_3$",
 '2h-pcy2tbu': "P(Cy)$_2$(t-Bu)",
 '2g-pcy3': "P(Cy)$_3$",
 '3b-phjohnphos': "PhJohnPhos",
 '3c-phdavephos': "PhDavePhos",
 '2e-pipr3': "P(i-Pr)$_3$",
 '2d-potol3': "P(o-tol)$_3$",
 '2c-pph3': "P(Ph)$_3$",
 '2i-ptbu2cy': "P(t-Bu)$_2$(Cy)",
 '2j-ptbu3': "P(t-Bu)$_3$",
 '3h-ruphos': "RuPhos",
 '3g-sphos': "SPhos",
 '3k-tbujohnphos': "JohnPhos",
 '3o-tbubrettphos': "tBuBrettPhos",
 '3m-tbudavephos': "tBuDavePhos",
 '3l-tbumephos': "tBuMePhos",
 '3n-tbuxphos': "tBuXPhos",
 '3i-xphos': "XPhos",   
 '9c-etjohnphos': "EtJohnPhos",
 '9d-mejohnphos': "MeJohnPhos",
 '9a-cbujohnphos': 'CbuJohnPhos',
 '9b-cptjohnphos': 'CptJohnPhos',
 '9e-pcbu3': 'P(Cbu)$_3$',     
 '9f-pcpr3': 'P(Cpr)$_3$',
 '9h-pme3': 'P(Me)$_3$',
 '9g-pet3': 'P(Et)$_3$',
}

structure_dict = {'main': list(proper_names_dict.keys()),
                  'altpdb': ['3q-adjohnphos', '3r-adbrettphos', '3j-brettphos', '3d-cyjohnphos', '3e-cymephos', '3f-davephos',
                             '3a-iprjohnphos', '3p-me4tbuxphos', '2k-pad3', '2f-pcpt3', '2h-pcy2tbu', '2g-pcy3', '3b-phjohnphos',
                             '3c-phdavephos', '2e-pipr3', '2d-potol3', '2c-pph3', '2i-ptbu2cy', '2j-ptbu3', '3h-ruphos',
                            '3g-sphos', '3k-tbujohnphos', '3o-tbubrettphos', '3m-tbudavephos', '3l-tbumephos', '3n-tbuxphos',
                            '3i-xphos',],
                  'xa': ['3n-tbuxphos', '3o-tbubrettphos', '3p-me4tbuxphos'],
                  'xb': ['3n-tbuxphos', '3o-tbubrettphos', '3p-me4tbuxphos'],          
                  'xaxb': ['3n-tbuxphos', '3o-tbubrettphos', '3p-me4tbuxphos'],
                  'ya': ['3j-brettphos', '3d-cyjohnphos', '3i-xphos', '3a-iprjohnphos', '3r-adbrettphos', '3n-tbuxphos'],
                  'yb': ['3j-brettphos', '3d-cyjohnphos', '3i-xphos', '3a-iprjohnphos', '3r-adbrettphos', '3n-tbuxphos'],
                  'yayb': ['3j-brettphos', '3d-cyjohnphos', '3i-xphos', '3a-iprjohnphos', '3r-adbrettphos', '3n-tbuxphos'],
                  'p1': ['9h-pme3', '9g-pet3', '2e-pipr3', '9f-pcpr3', '9e-pcbu3', '2f-pcpt3', '2g-pcy3', '2c-pph3', '3d-cyjohnphos',
                          '3a-iprjohnphos', '9a-cbujohnphos', '9b-cptjohnphos', '9c-etjohnphos', '9d-mejohnphos', '3k-tbujohnphos'],
                  'alternative': ['2g-pcy3', '2j-ptbu3', '3d-cyjohnphos', '3k-tbujohnphos'],
                  "ob": ['3n-tbuxphos', '3o-tbubrettphos', '3p-me4tbuxphos'],
                  "hc": ['2g-pcy3', '2j-ptbu3']}

## Processing calculation results

In [3]:
df = pd.read_csv("Goodvibes_output.csv", skiprows=23)
df = df.tail(-1)
df = df.head(-1)
df = df[['   Structure', 'qh-G(T)_SPC', 'im']]
df.rename(columns={'   Structure': 'filename'}, inplace=True)
df['filename'] = df.apply(lambda row: row['filename'].split()[1], axis=1)
df[['group', 'structure']] = df.apply(lambda row: split_filename(row['filename']), axis=1, result_type='expand')


In [4]:
base_reactants_df = df[df['group'] == "base-reactants"].copy()
base_catalysis_df = df[df['group'] == "base-catalysis"].copy()
main_df = df[((~df['group'].isin(['base-catalysis', 'base-reactants', "unlig-pdoh2"])) & (df['structure'] != '84-lig') & (~df['structure'].str.contains('ref')))].copy()
ligands_df = df[df['structure'] == '84-lig'].copy()
unlig_df =  df[df['group'] == "unlig-pdoh2"].copy()

In [5]:
main_df['equalized_gibbs'] = main_df.apply(lambda row: equalize_reference(row, row['group']), axis=1)
main_df['reference_gibbs'] = main_df.apply(lambda row: (row['equalized_gibbs'] - get_reference_energy(main_df, row['group']))*2625.5/4.184, axis=1)

## Create microkinetic pathways

In [6]:
diff_barrier = 3.4663630747609946
type_list = ['main', 'xa', 'xb', 'xaxb', 'ya', 'yb', 'yayb', 'p1', 'ob', 'hc']

for type_name in type_list:
    int_list, ts_list, rxt_pdt_list, diffusion_ts_list, xform_dict = get_main_microkinetics_transformations(type_name=type_name)

    for ligand in structure_dict[type_name]:
        os.makedirs(f"microkinetics/{type_name}-{ligand}", exist_ok=True)
        print(f"{type_name}-{ligand}")
        current_df = main_df[main_df['group'] == ligand][['structure', 'qh-G(T)_SPC']].copy()
        current_df = pd.concat([current_df, pd.DataFrame([{'structure': '99-l2pd2oh4', 'qh-G(T)_SPC': current_df[current_df['structure'] == '00-lpdoh2']['qh-G(T)_SPC'].values[0]}])], ignore_index=True)
        current_df.loc[current_df['structure'] == '00-lpdoh2', 'qh-G(T)_SPC'] /= 2

        if type_name == 'hc':
            current_df.loc[current_df['structure'] == '70-c2dim', 'qh-G(T)_SPC'] /= 2
        int_df = current_df[(~current_df['structure'].str.contains('ts')) & (current_df['structure'].isin(int_list))].copy()
        
        for rxt in rxt_pdt_list:
            if rxt == "ligand":
                int_df = pd.concat([int_df, pd.DataFrame([{'structure': rxt, 'qh-G(T)_SPC': get_ligand_energy(ligands_df, ligand)}])], ignore_index=True)
            elif rxt == "pre-catalyst-dimer":
                int_df = pd.concat([int_df, pd.DataFrame([{'structure': rxt, 'qh-G(T)_SPC': get_precatalyst_energy(current_df, ligand)}])], ignore_index=True)
            elif rxt == "pre-catalyst-monomer":
                int_df = pd.concat([int_df, pd.DataFrame([{'structure': rxt, 'qh-G(T)_SPC': get_precatalyst_energy(current_df, ligand)/2}])], ignore_index=True)
            else:
                int_df = pd.concat([int_df, pd.DataFrame([{'structure': rxt, 'qh-G(T)_SPC': get_reactant_energy(rxt)}])], ignore_index=True)
        

        int_df.to_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-intermediates.csv", index=False)

        xform_df = pd.DataFrame(columns=['I1', 'I2', 'P1', 'P2', 'Energy', 'Backwards', 'Flag', 'Ref_L', 'Ref_R'])
        xform_df["Backwards"] = xform_df["Backwards"].astype(bool)

        for ts, xform in xform_dict.items():
            energy_l, energy_r = get_left_right_barrier_reference_energies(int_df, xform)
            if ts in diffusion_ts_list:
                ts_energy = max(energy_l, energy_r)
            else:
                ts_energy = get_energy(current_df, ts)

            xform_df = pd.concat([xform_df, pd.DataFrame([{'I1': xform[0], 'I2': xform[1], 'P1': xform[2], 'P2': xform[3], 
                                                        'Energy': ts_energy, 'Backwards': xform[4], 'Flag': ts, 
                                                        'Ref_L': (ts_energy-energy_l)*2625.5/4.184, 
                                                        'Ref_R': (ts_energy-energy_r)*2625.5/4.184,}],
                                                        )])

        xform_df.to_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-transformations.csv", index=False)    


main-3q-adjohnphos
main-3r-adbrettphos
main-3j-brettphos
main-3d-cyjohnphos
main-3e-cymephos
main-3f-davephos
main-3a-iprjohnphos
main-3p-me4tbuxphos
main-2k-pad3
main-2f-pcpt3
main-2h-pcy2tbu
main-2g-pcy3
main-3b-phjohnphos
main-3c-phdavephos
main-2e-pipr3
main-2d-potol3
main-2c-pph3
main-2i-ptbu2cy
main-2j-ptbu3
main-3h-ruphos
main-3g-sphos
main-3k-tbujohnphos
main-3o-tbubrettphos
main-3m-tbudavephos
main-3l-tbumephos
main-3n-tbuxphos
main-3i-xphos
main-9c-etjohnphos
main-9d-mejohnphos
main-9a-cbujohnphos
main-9b-cptjohnphos
main-9e-pcbu3
main-9f-pcpr3
main-9h-pme3
main-9g-pet3
xa-3n-tbuxphos
xa-3o-tbubrettphos
xa-3p-me4tbuxphos
xb-3n-tbuxphos
xb-3o-tbubrettphos
xb-3p-me4tbuxphos
xaxb-3n-tbuxphos
xaxb-3o-tbubrettphos
xaxb-3p-me4tbuxphos
ya-3j-brettphos
ya-3d-cyjohnphos
ya-3i-xphos
ya-3a-iprjohnphos
ya-3r-adbrettphos
ya-3n-tbuxphos
yb-3j-brettphos
yb-3d-cyjohnphos
yb-3i-xphos
yb-3a-iprjohnphos
yb-3r-adbrettphos
yb-3n-tbuxphos
yayb-3j-brettphos
yayb-3d-cyjohnphos
yayb-3i-xphos
yayb-3a-

## Plotting reaction profile diagrams

In [7]:
def interpolate(df, line, num=1000):
    sub_df = df[df['line'] == line]
    x_min = min(sub_df['point'])
    x_max = max(sub_df['point'])
    x_interp = np.linspace(x_min, x_max, num)
    # Turning point at energy minima/maxima
    spline_anchors = [(row['point'], [row['reference_gibbs'], 0]) for _ , row in sub_df.iterrows()]
    #print(spline_anchors)
    spline = BPoly.from_derivatives(xi=sub_df['point'], yi=[[row['reference_gibbs'], 0] for _ , row in sub_df.iterrows()])
    return x_interp, spline(x_interp)

In [8]:
base_diffusion_consideration_dict = {"C2 // H$_2$O": ["C2", "C2 + H$_2$O"],
                       "C3 // B(OH)$_3$": ["C3 + B(OH)$_3$", "C3"],
                       "C4-PDT": ['C4', 'PDT'],
                       "T2 // H$_2$O": ["T2", "T2 + H$_2$O"],
                       "T3 // B(OH)$_3$": ["T3 + B(OH)$_3$", "T3"],
                       'T4-PDT': ["T4", "PDT"],
                       "Pd-RXT": ["[LPd(OH)$_2$]$_2$", "RXT",],
                        }

pdt_energy = (get_reactant_energy('naphh') + get_reactant_energy('boh3') - get_reactant_energy('naphboh2') - get_reactant_energy('h2o'))*2625.5/4.184

type_list=['main']
for type_name in type_list:

    diffusion_consideration_dict = dict(base_diffusion_consideration_dict)

    if type_name == 'p1':
        diffusion_consideration_dict["T2 // L"] = ["T2", "T2 + L"]
        diffusion_consideration_dict["T3 // L"] = ["T3", "T3 + L"]
        diffusion_consideration_dict["P1 // B(OH)$_3$"] = ["P1 + B(OH)$_3$", "P1"]
        diffusion_consideration_dict["P1 // H$_2$O"] = ["P1 + H$_2$O", "P1"]

    if type_name == 'ob':
        diffusion_consideration_dict["T3OB + R-H // T3OB"] = ["T3OB + R-H", "T3OB"]
        diffusion_consideration_dict["T3OB // T3OB + H$_2$O"] = ["T3OB", "T3OB + H$_2$O"]
        diffusion_consideration_dict["T3OB + H$_2$O // T3OB + 2H$_2$O"] = ["T3OB + H$_2$O", "T3OB + 2H$_2$O"]
        diffusion_consideration_dict["T4OB + H$_2$O // T4OB"] = ["T4OB + H$_2$O", "T4OB"]
        diffusion_consideration_dict["T4OB // PDT"] = ["T4OB", "PDT"]

    if type_name == 'hc':
        diffusion_consideration_dict["C2$_{dimer}$ // C2"] = ["C2$_{dimer}$", "C2"]
        diffusion_consideration_dict["C2 // C2 + NaphB(OH)$_2$"] = ["C2", "C2 + NaphB(OH)$_2$"]
        diffusion_consideration_dict["HC1 + B(OH)$_3$ // HC1"] = ["HC1 + B(OH)$_3$", "HC1"]
        diffusion_consideration_dict["HC3 // HC4 + B(OH)$_3$"] = ["HC3", "HC4 + B(OH)$_3$"]
        diffusion_consideration_dict["LPd(0)B(OH)$_3$ // LPd(0) + B(OH)$_3$"] = ["LPd(0)B(OH)$_3$", "LPd(0) + B(OH)$_3$"]        
    for ligand in structure_dict[type_name]:
        print(f"{type_name}-{ligand}")
        os.makedirs(f"microkinetics/{type_name}-{ligand}", exist_ok=True)
        current_df = main_df[main_df['group'] == ligand][['structure', 'reference_gibbs']].copy()
        current_df['label_name'] = current_df.apply(lambda row: nice_structure_names[row['structure']], axis=1)
        current_df = pd.concat([current_df, pd.DataFrame([{'structure': 'pdt', 'reference_gibbs': pdt_energy, 'label_name': 'PDT'}])], ignore_index=True)
        for diffusion, consideration in diffusion_consideration_dict.items():
            #print(consideration)
            energy = max(get_reference_gibbs_energy(current_df, consideration[0]), get_reference_gibbs_energy(current_df, consideration[1]))+diff_barrier
            current_df = pd.concat([current_df, pd.DataFrame([{'structure': 'diffusion', 'reference_gibbs': energy, 'label_name': diffusion}])], ignore_index=True)  
        try:
            template = pd.read_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot.csv")
            template = pd.merge(template, current_df[['label_name', 'reference_gibbs']], on='label_name', how='left', suffixes=('_old', ""))
            template = template.drop('reference_gibbs_old', axis=1)
            template.sort_values(by=['line', 'point'], ascending=[True, True], inplace=True)
            template.to_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot.csv", index=False)
        except FileNotFoundError:
            template = pd.read_csv(f"templates/rpd_template_{type_name}.csv")
            template = pd.merge(template, current_df, on='label_name')
            template.to_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot.csv", index=False)

        fig, ax = plt.subplots(figsize=(6,3), dpi=300)
        for line in set(template['line'].values):
            curr_line_df = template[template['line'] == line].copy().sort_values(by='point').reset_index(drop=True)
            x_solid, y_solid = interpolate(curr_line_df, line=line)

            for idx, point in curr_line_df.iterrows():
                if point['is_point'] == True:
                    ax.scatter(point['point'], point['reference_gibbs'], color=point['point_color'], marker=point['point_marker'], s=10)
                if point['is_TS'] == True:
                    # Locate triangle marker index
                    idx_0 = np.where(x_solid >= curr_line_df.at[idx-1, 'point'])[0][0]
                    idx_1 = np.where(x_solid >= point['point'])[0][0]
                    idx_2 = np.where(x_solid >= curr_line_df.at[idx+1, 'point'])[0][0]
                    ax.plot(x_solid[idx_0:idx_2], y_solid[idx_0:idx_2], color=curr_line_df['line_color'].values[0], 
                            zorder=-1, linewidth=1, linestyle='--' if point['point_marker'] == 'x' or point['point_marker'] == 'v' else '-')
                
                if point['is_label'] == True:
                    label_text = f"{point['label_name']}" f"\n{point['reference_gibbs']:.1f}"

                    ax.annotate(label_text, xy=(point['point'], point['reference_gibbs']), 
                                ha='center', color=point['point_color'], va=point['vertical_alignment'], 
                                fontsize=5, xytext=(point['point'], point['reference_gibbs']+point['vertical_offset']))
                
            
                    
        ax.spines[['right', 'top', 'bottom']].set_visible(False)           
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        plt.tick_params(left = False,bottom=False) 
        ax.set_xlim(left=-1.5)    
        ax.plot(-1.5, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False, markersize=2)

        ax.set_ylabel("$\Delta G$ / $kcal$ $mol^{-1}$", size=8)
        if f"{type_name}-{ligand}" == "p1-3k-tbujohnphos":
            ax.set_ylim(-30, 45)
        elif f"{type_name}-{ligand}" == "yayb-3n-tbuxphos" or f"{type_name}-{ligand}" == "yayb-3r-adbrettphos":
            ax.set_ylim(-30, 40)
        elif type_name == 'ob' or type_name == 'xaxb':
            ax.set_ylim(-30, 30)
        elif type_name == 'hc':
            ax.set_ylim(-65, 30)
        else:
            ax.set_ylim(-45, 30)
        plt.title(f"L = {proper_names_dict[ligand]}", size=8)
        plt.savefig(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot.png", facecolor='white', transparent=False, bbox_inches='tight')
        plt.close()

main-3q-adjohnphos
main-3r-adbrettphos
main-3j-brettphos
main-3d-cyjohnphos
main-3e-cymephos
main-3f-davephos
main-3a-iprjohnphos
main-3p-me4tbuxphos
main-2k-pad3
main-2f-pcpt3
main-2h-pcy2tbu
main-2g-pcy3
main-3b-phjohnphos
main-3c-phdavephos
main-2e-pipr3
main-2d-potol3
main-2c-pph3
main-2i-ptbu2cy
main-2j-ptbu3
main-3h-ruphos
main-3g-sphos
main-3k-tbujohnphos
main-3o-tbubrettphos
main-3m-tbudavephos
main-3l-tbumephos
main-3n-tbuxphos
main-3i-xphos
main-9c-etjohnphos
main-9d-mejohnphos
main-9a-cbujohnphos
main-9b-cptjohnphos
main-9e-pcbu3
main-9f-pcpr3
main-9h-pme3
main-9g-pet3


## Barrier Tuning to Parity Prediction Example Plots

In [9]:
for type_name in ['main']:
    for ligand in ['2g-pcy3', '3d-cyjohnphos']:
        diffusion_consideration_dict = dict(base_diffusion_consideration_dict)
        print(f"{type_name}-{ligand}")
        current_df = main_df[main_df['group'] == ligand][['structure', 'reference_gibbs']].copy()
        current_df['label_name'] = current_df.apply(lambda row: nice_structure_names[row['structure']], axis=1)
        current_df = pd.concat([current_df, pd.DataFrame([{'structure': 'pdt', 'reference_gibbs': pdt_energy, 'label_name': 'PDT'}])], ignore_index=True)
        for diffusion, consideration in diffusion_consideration_dict.items():
            energy = max(get_reference_gibbs_energy(current_df, consideration[0]), get_reference_gibbs_energy(current_df, consideration[1]))+diff_barrier
            current_df = pd.concat([current_df, pd.DataFrame([{'structure': 'diffusion', 'reference_gibbs': energy, 'label_name': diffusion}])], ignore_index=True)
            
        template = pd.read_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot-btpp.csv")

        fig, ax = plt.subplots(figsize=(6,3), dpi=300)
        for line in set(template['line'].values):
            curr_line_df = template[template['line'] == line].copy().sort_values(by='point').reset_index(drop=True)
            x_solid, y_solid = interpolate(curr_line_df, line=line)

            for idx, point in curr_line_df.iterrows():
                if point['is_point'] == True:
                    ax.scatter(point['point'], point['reference_gibbs'], color=point['point_color'], marker=point['point_marker'], s=10)
                if point['is_TS'] == True:
                    # Locate triangle marker index
                    idx_0 = np.where(x_solid >= curr_line_df.at[idx-1, 'point'])[0][0]
                    idx_1 = np.where(x_solid >= point['point'])[0][0]
                    idx_2 = np.where(x_solid >= curr_line_df.at[idx+1, 'point'])[0][0]
                    ax.plot(x_solid[idx_0:idx_2], y_solid[idx_0:idx_2], color=curr_line_df['line_color'].values[0], 
                            zorder=-1, linewidth=1, linestyle='--' if point['point_marker'] == 'x' else '-')
                
                if point['is_label'] == True:
                    label_text = f"{point['label_name']}" f"\n{point['reference_gibbs']:.1f}"

                    ax.annotate(label_text, xy=(point['point'], point['reference_gibbs']), 
                                ha='center', color=point['point_color'], va=point['vertical_alignment'], 
                                fontsize=5, xytext=(point['point'], point['reference_gibbs']+point['vertical_offset']))
                
            
                    
        ax.spines[['right', 'top', 'bottom']].set_visible(False)           
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        plt.tick_params(left = False,bottom=False) 
        ax.set_xlim(left=-1.5)
        ax.plot(-1.5, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False, markersize=2)
        ax.set_ylabel("$\Delta G$ / $kcal$ $mol^{-1}$", size=8)
        ax.set_ylim(-45, 30)
        plt.title(f"L = {proper_names_dict[ligand]}", size=8)
        plt.savefig(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot-btpp.png", facecolor='white', transparent=False, bbox_inches='tight')
        plt.close()

main-2g-pcy3
main-3d-cyjohnphos


## Theoretical Plot

In [11]:
template = pd.read_csv(f"templates/rpd_template_theoretical.csv")

fig, ax = plt.subplots(figsize=(6,3), dpi=300)
for line in set(template['line'].values):
    curr_line_df = template[template['line'] == line].copy().sort_values(by='point').reset_index(drop=True)
    x_solid, y_solid = interpolate(curr_line_df, line=line)

    for idx, point in curr_line_df.iterrows():
        if point['is_point'] == True:
            ax.scatter(point['point'], point['reference_gibbs'], color=point['point_color'], marker=point['point_marker'], s=10)
        if point['is_TS'] == True:
            # Locate triangle marker index
            idx_0 = np.where(x_solid >= curr_line_df.at[idx-1, 'point'])[0][0]
            idx_1 = np.where(x_solid >= point['point'])[0][0]
            idx_2 = np.where(x_solid >= curr_line_df.at[idx+1, 'point'])[0][0]
            ax.plot(x_solid[idx_0:idx_2], y_solid[idx_0:idx_2], color=curr_line_df['line_color'].values[0], 
                    zorder=-1, linewidth=1, linestyle='--' if point['point_marker'] == 'x' else '-')
        
        if point['is_label'] == True:
            label_text = f"{point['label_name']}" f"\n{point['reference_gibbs']:.1f}"

            ax.annotate(label_text, xy=(point['point'], point['reference_gibbs']), 
                        ha='center', color=point['point_color'], va=point['vertical_alignment'], 
                        fontsize=5, xytext=(point['point'], point['reference_gibbs']+point['vertical_offset']))
        
    
            
ax.spines[['right', 'top', 'bottom']].set_visible(False)           
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.tick_params(left = False,bottom=False) 
ax.set_xlim(left=-1)
ax.plot(-1, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False, markersize=2)
ax.set_ylabel("$\Delta G$ / $kcal$ $mol^{-1}$", size=8)
ax.set_ylim(-45, 30)
plt.title(f"L = Theoretical", size=8)
plt.savefig(f"theoretical-rpd.png", facecolor='white', transparent=False, bbox_inches='tight')
plt.close()

## Setup for Error Curve analysis

In [12]:
exp_yield_dict = {'2c-pph3': {'exp_yield': 0.008, 'exp_error': 0.0001},
 '2d-potol3': {'exp_yield': 0.647, 'exp_error': 0.0333},
 '2e-pipr3': {'exp_yield': 0.035, 'exp_error': 0.0131},
 '2f-pcpt3': {'exp_yield': 0.04, 'exp_error': 0.0091},
 '2g-pcy3': {'exp_yield': 0.036, 'exp_error': 0.028},
 '2h-pcy2tbu': {'exp_yield': 0.1, 'exp_error': 0.0255},
 '2i-ptbu2cy': {'exp_yield': 0.322, 'exp_error': 0.0127},
 '2j-ptbu3': {'exp_yield': 0.907, 'exp_error': 0.0186},
 '2k-pad3': {'exp_yield': 0.43, 'exp_error': 0.0466},
 '3a-iprjohnphos': {'exp_yield': 0.085, 'exp_error': 0.0431},
 '3b-phjohnphos': {'exp_yield': 0.186, 'exp_error': 0.0007},
 '3c-phdavephos': {'exp_yield': 0.089, 'exp_error': 0.0},
 '3d-cyjohnphos': {'exp_yield': 0.106, 'exp_error': 0.0057},
 '3e-cymephos': {'exp_yield': 0.154, 'exp_error': 0.0276},
 '3f-davephos': {'exp_yield': 0.209, 'exp_error': 0.0323},
 '3g-sphos': {'exp_yield': 0.15, 'exp_error': 0.0029},
 '3h-ruphos': {'exp_yield': 0.11, 'exp_error': 0.0175},
 '3i-xphos': {'exp_yield': 0.124, 'exp_error': 0.0278},
 '3j-brettphos': {'exp_yield': 0.063, 'exp_error': 0.0277},
 '3k-tbujohnphos': {'exp_yield': 0.7, 'exp_error': 0.0155},
 '3l-tbumephos': {'exp_yield': 0.448, 'exp_error': 0.0087},
 '3m-tbudavephos': {'exp_yield': 0.855, 'exp_error': 0.0277},
 '3n-tbuxphos': {'exp_yield': 0.829, 'exp_error': 0.0117},
 '3o-tbubrettphos': {'exp_yield': 0.66, 'exp_error': 0.0833},
 '3p-me4tbuxphos': {'exp_yield': 0.343, 'exp_error': 0.0884},
 '3q-adjohnphos': {'exp_yield': 0.257, 'exp_error': 0.0009},
 '3r-adbrettphos': {'exp_yield': 0.252, 'exp_error': 0.0105},
 '9a-cbujohnphos': {'exp_yield': np.nan, 'exp_error': np.nan},
 '9b-cptjohnphos': {'exp_yield': np.nan, 'exp_error': np.nan},
 '9c-etjohnphos': {'exp_yield': np.nan, 'exp_error': np.nan},
 '9d-mejohnphos': {'exp_yield': np.nan, 'exp_error': np.nan},
 '9e-pcbu3': {'exp_yield': np.nan, 'exp_error': np.nan},
 '9f-pcpr3': {'exp_yield': np.nan, 'exp_error': np.nan},
 '9g-pet3': {'exp_yield': np.nan, 'exp_error': np.nan},
 '9h-pme3': {'exp_yield': np.nan, 'exp_error': np.nan}}

In [13]:
pivoted_df = main_df.pivot(index='group', columns='structure', values='reference_gibbs')
pivoted_df = pivoted_df.reset_index()

pivoted_df['exp_yield'] = pivoted_df.apply(lambda row: exp_yield_dict[row['group']]['exp_yield'], axis=1)
pivoted_df['exp_error'] = pivoted_df.apply(lambda row: exp_yield_dict[row['group']]['exp_error'], axis=1)

pivoted_df['max_C_TM'] = pivoted_df[['02-ts-rxt-c1', '04-ts-c1-c2']].max(axis=1)
pivoted_df['max_C_TM_name'] = pivoted_df[['02-ts-rxt-c1', '04-ts-c1-c2']].idxmax(axis=1)
pivoted_df['max_T_uncorr'] = pivoted_df[['12-ts-rxt-t1', '14-ts-t1-t2', '17-ts-t2-t3', '20-ts-t3-t4']].max(axis=1)
pivoted_df['max_T_uncorr_name'] = pivoted_df[['12-ts-rxt-t1', '14-ts-t1-t2', '17-ts-t2-t3', '20-ts-t3-t4']].idxmax(axis=1)
pivoted_df['max_overall_uncorr_name'] = pivoted_df[['02-ts-rxt-c1', '04-ts-c1-c2', '12-ts-rxt-t1', '14-ts-t1-t2', '17-ts-t2-t3', '20-ts-t3-t4']].idxmax(axis=1)
pivoted_df['ddG_uncorr'] = pivoted_df['max_C_TM'] - pivoted_df['max_T_uncorr']

for h2o_eq in [1.0, 3.5, 5.0]:
    pivoted_df[f'17-ts-t2-t3-corr_{h2o_eq}'] = pivoted_df['17-ts-t2-t3'] + get_second_order_barrier_correction(h2o_eq*0.02, 333.15)
    pivoted_df[f'max_T_corr_{h2o_eq}'] = pivoted_df[['12-ts-rxt-t1', '14-ts-t1-t2', f'17-ts-t2-t3-corr_{h2o_eq}', '20-ts-t3-t4']].max(axis=1)
    pivoted_df[f'max_T_corr_{h2o_eq}_name'] = pivoted_df[['12-ts-rxt-t1', '14-ts-t1-t2', f'17-ts-t2-t3-corr_{h2o_eq}', '20-ts-t3-t4']].idxmax(axis=1)
    pivoted_df[f'max_overall_corr_{h2o_eq}_name'] = pivoted_df[['02-ts-rxt-c1', '04-ts-c1-c2', '12-ts-rxt-t1', '14-ts-t1-t2', f'17-ts-t2-t3-corr_{h2o_eq}', '20-ts-t3-t4']].idxmax(axis=1)
    pivoted_df[f'ddG_corr_{h2o_eq}'] = pivoted_df['max_C_TM'] - pivoted_df[f'max_T_corr_{h2o_eq}']


pivoted_df.to_csv("all_barriers.csv", index=False)


## Setup for Microkinetics

In [14]:
microkinetics_of_interest = []

for main in structure_dict['main']:
    microkinetics_of_interest.append(f"main-{main}")

for xa in structure_dict['xa']:
    microkinetics_of_interest.append(f"xa-{xa}")

for xb in structure_dict['xb']:
    microkinetics_of_interest.append(f"xb-{xb}")

for p1 in structure_dict['p1']:
    microkinetics_of_interest.append(f"p1-{p1}")

microkinetics_of_interest.append("yayb-3j-brettphos")

base_mk_params = pd.DataFrame(microkinetics_of_interest, columns=["microkinetic_name"])
base_mk_params['microkinetic_type'] = base_mk_params.apply(lambda row: row['microkinetic_name'].split('-')[0], axis=1)
base_mk_params['ligand_name'] = base_mk_params.apply(lambda row: row['microkinetic_name'].split('-')[1] + '-' + row['microkinetic_name'].split('-')[2], axis=1)
base_mk_params['exp_yield'] = base_mk_params.apply(lambda row: exp_yield_dict[row['ligand_name']]['exp_yield'], axis=1)
base_mk_params['exp_error'] = base_mk_params.apply(lambda row: exp_yield_dict[row['ligand_name']]['exp_error'], axis=1)

pivoted_df['min_of_max_barrier'] = pivoted_df[['max_C_TM', 'max_T_corr_3.5']].min(axis=1)

base_mk_params = pd.merge(base_mk_params, pivoted_df[['group', 'min_of_max_barrier']], left_on='ligand_name', right_on='group')
base_mk_params.sort_values(by=['microkinetic_name', 'microkinetic_type'], ascending=[True, True], inplace=True)
base_mk_params = base_mk_params.drop(columns=['group']).reset_index(drop=True)

base_mk_params.to_csv("base_microkinetic_parameters.csv", index=False)


## Base Catalysis

In [15]:
base_catalysis_naphboh3_dict = {"reference" : get_reactant_energy('naphboh3') + get_reactant_energy('naphboh2') + get_reactant_energy('h2o')}
base_catalysis_naphboh3_dict["[NaphB(OH)$_3$]$^-$"] = (get_reactant_energy('naphboh3') + get_reactant_energy('naphboh2') + get_reactant_energy('h2o') - base_catalysis_naphboh3_dict["reference"])*2625.5/4.184
base_catalysis_naphboh3_dict["TS0"] = (get_energy(base_catalysis_df, '00-ts-naphboh3-ts0') + get_reactant_energy('naphboh2') + get_reactant_energy('h2o') - base_catalysis_naphboh3_dict["reference"])*2625.5/4.184
base_catalysis_naphboh3_dict["Naph-H + [OB(OH)$_2$]$^-$"] = (get_reactant_energy('naphh') + get_reactant_energy('oboh2') + get_reactant_energy('naphboh2') + get_reactant_energy('h2o') - base_catalysis_naphboh3_dict["reference"])*2625.5/4.184
base_catalysis_naphboh3_dict["Naph-H + [B(OH)$_4$]$^-$"] = (get_reactant_energy('naphh') + get_reactant_energy('boh4') + get_reactant_energy('naphboh2') - base_catalysis_naphboh3_dict["reference"])*2625.5/4.184
base_catalysis_naphboh3_dict["[NaphB(OH)$_3$]$^-$ + H$_2$O"] = (get_energy(base_catalysis_df, '01-naphboh3-h2o') + get_reactant_energy('naphboh2') - base_catalysis_naphboh3_dict["reference"])*2625.5/4.184
base_catalysis_naphboh3_dict["TS1"] = (get_energy(base_catalysis_df, '02-ts-naphboh3-ts1') + get_reactant_energy('naphboh2') - base_catalysis_naphboh3_dict["reference"])*2625.5/4.184
base_catalysis_naphboh3_dict["Naph-H + B(OH)$_3$ + [NaphB(OH)$_3$]$^-$"] = (get_reactant_energy("naphh") + get_reactant_energy('boh3') + get_reactant_energy('naphboh3') - base_catalysis_naphboh3_dict["reference"])*2625.5/4.184

base_catalysis_naphboh3_dict["Naph-H + [OB(OH)$_2$]$^-$ // Naph-H + [B(OH)$_4$]$^-$"] = max(base_catalysis_naphboh3_dict["Naph-H + [OB(OH)$_2$]$^-$"], base_catalysis_naphboh3_dict["Naph-H + [B(OH)$_4$]$^-$"])+diff_barrier
base_catalysis_naphboh3_dict["[NaphB(OH)$_3$]$^-$ // [NaphB(OH)$_3$]$^-$ + H$_2$O"] = max(base_catalysis_naphboh3_dict["[NaphB(OH)$_3$]$^-$"], base_catalysis_naphboh3_dict["[NaphB(OH)$_3$]$^-$ + H$_2$O"])+diff_barrier
base_catalysis_naphboh3_dict["Naph-H + [B(OH)$_4$]$^-$ // Naph-H + B(OH)$_3$ + [NaphB(OH)$_3$]$^-$"] = max(base_catalysis_naphboh3_dict["Naph-H + [B(OH)$_4$]$^-$"], base_catalysis_naphboh3_dict["Naph-H + B(OH)$_3$ + [NaphB(OH)$_3$]$^-$"])+diff_barrier

In [16]:
base_catalysis_naphbpin_dict = {"reference" : get_reactant_energy('naphbpinoh') + get_reactant_energy('naphbpin') + get_reactant_energy('h2o')}
base_catalysis_naphbpin_dict["[NaphBpin(OH)]$^-$"] = (get_reactant_energy('naphbpinoh') + get_reactant_energy('naphbpin') + get_reactant_energy('h2o') - base_catalysis_naphbpin_dict["reference"])*2625.5/4.184
base_catalysis_naphbpin_dict["TS0"] = (get_energy(base_catalysis_df, '10-ts-naphbpinoh-ts0') + get_reactant_energy('naphbpin') + get_reactant_energy('h2o') - base_catalysis_naphbpin_dict["reference"])*2625.5/4.184
base_catalysis_naphbpin_dict["Naph-H + [OBpin]$^-$"] = (get_reactant_energy('naphh') + get_reactant_energy('obpin') + get_reactant_energy('naphbpin') + get_reactant_energy('h2o') - base_catalysis_naphbpin_dict["reference"])*2625.5/4.184
base_catalysis_naphbpin_dict["Naph-H + [Bpin(OH)$_2$]$^-$"] = (get_reactant_energy('naphh') + get_reactant_energy('bpinoh2') + get_reactant_energy('naphbpin') - base_catalysis_naphbpin_dict["reference"])*2625.5/4.184
base_catalysis_naphbpin_dict["[NaphBpin(OH)]$^-$ + H$_2$O"] = (get_energy(base_catalysis_df, '11-naphbpinoh-h2o') + get_reactant_energy('naphbpin') - base_catalysis_naphbpin_dict["reference"])*2625.5/4.184
base_catalysis_naphbpin_dict["TS1"] = (get_energy(base_catalysis_df, '12-ts-naphbpinoh-ts1') + get_reactant_energy('naphbpin') - base_catalysis_naphbpin_dict["reference"])*2625.5/4.184
base_catalysis_naphbpin_dict["Naph-H + Bpin(OH) + [NaphBpin(OH)]$^-$"] = (get_reactant_energy("naphh") + get_reactant_energy('bpinoh') + get_reactant_energy('naphbpinoh') - base_catalysis_naphbpin_dict["reference"])*2625.5/4.184


base_catalysis_naphbpin_dict["Naph-H + [OBpin]$^-$ // Naph-H + [Bpin(OH)$_2$]$^-$"] = max(base_catalysis_naphbpin_dict["Naph-H + [OBpin]$^-$"], base_catalysis_naphbpin_dict["Naph-H + [Bpin(OH)$_2$]$^-$"])+diff_barrier
base_catalysis_naphbpin_dict["[NaphBpin(OH)]$^-$ // [NaphBpin(OH)]$^-$ + H$_2$O"] = max(base_catalysis_naphbpin_dict["[NaphBpin(OH)]$^-$"], base_catalysis_naphbpin_dict["[NaphBpin(OH)]$^-$ + H$_2$O"])+diff_barrier
base_catalysis_naphbpin_dict["Naph-H + [Bpin(OH)$_2$]$^-$ // Naph-H + Bpin(OH) + [NaphBpin(OH)]$^-$"] = max(base_catalysis_naphbpin_dict["Naph-H + [Bpin(OH)$_2$]$^-$"], base_catalysis_naphbpin_dict["Naph-H + Bpin(OH) + [NaphBpin(OH)]$^-$"])+diff_barrier

In [17]:
for type_name in ['naphboh3', 'naphbpin']:
        os.makedirs(f"microkinetics/basecat-{type_name}", exist_ok=True)
        template = pd.read_csv(f"templates/rpd_template_basecat_{type_name}.csv")
        if type_name == 'naphboh3':
            nice_name = "NaphB(OH)$_2$"
            template['reference_gibbs'] = template.apply(lambda row: base_catalysis_naphboh3_dict[row['label_name']], axis=1)
        else:
            nice_name = "NaphBpin"
            template['reference_gibbs'] = template.apply(lambda row: base_catalysis_naphbpin_dict[row['label_name']], axis=1)
        template.to_csv(f"microkinetics/basecat-{type_name}/basecat-{type_name}-plot.csv", index=False)

        fig, ax = plt.subplots(figsize=(6,3), dpi=300)
        for line in set(template['line'].values):
            curr_line_df = template[template['line'] == line].copy().sort_values(by='point').reset_index(drop=True)
            x_solid, y_solid = interpolate(curr_line_df, line=line)

            for idx, point in curr_line_df.iterrows():
                if point['is_point'] == True:
                    ax.scatter(point['point'], point['reference_gibbs'], color=point['point_color'], marker=point['point_marker'], s=10)
                if point['is_TS'] == True:
                    # Locate triangle marker index
                    idx_0 = np.where(x_solid >= curr_line_df.at[idx-1, 'point'])[0][0]
                    idx_1 = np.where(x_solid >= point['point'])[0][0]
                    idx_2 = np.where(x_solid >= curr_line_df.at[idx+1, 'point'])[0][0]
                    ax.plot(x_solid[idx_0:idx_2], y_solid[idx_0:idx_2], color=curr_line_df['line_color'].values[0], 
                            zorder=-1, linewidth=1, linestyle='--' if point['point_marker'] == 'x' else '-')
                
                if point['is_label'] == True:
                    label_text = f"{point['label_name']}" f"\n{point['reference_gibbs']:.1f}"

                    ax.annotate(label_text, xy=(point['point'], point['reference_gibbs']), 
                                ha='center', color=point['point_color'], va=point['vertical_alignment'], 
                                fontsize=5, xytext=(point['point'], point['reference_gibbs']+point['vertical_offset']))
                
            
                    
        ax.spines[['right', 'top', 'bottom']].set_visible(False)           
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        plt.tick_params(left = False,bottom=False) 
        ax.set_xlim(left=-0.5)
        ax.plot(-0.5, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False, markersize=2)
        ax.set_ylabel("$\Delta G$ / $kcal$ $mol^{-1}$", size=8)
        ax.set_ylim(-30, 35)
        plt.title(f"Base-catalysed {nice_name}", size=8)
        plt.savefig(f"microkinetics/basecat-{type_name}/basecat-{type_name}-plot.png", facecolor='white', transparent=False, bbox_inches='tight')
        plt.close()

## Unligated Catalysis

In [18]:
def equalize_unlig_reference(row):
    if row['structure'] in ['00-murxt']:
        return row['qh-G(T)_SPC']/2 + get_energy(base_reactants_df, 'naphboh2') + get_energy(base_reactants_df, 'h2o')
    elif row['structure'] in ['01-rxt', '02-ts-rxt-int1', '03-int1', '04-ts-int1-int2', '05-int2',
                            ]:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'h2o')
    
    elif row['structure'] in ['06-int2-h2o', '07-ts-int2-int3', '08-int3-boh3',
                            ]:
        return row['qh-G(T)_SPC']
    elif row['structure'] in ['09-int3', '10-ts-int3-int4', '11-int4',
                            ]:
        return row['qh-G(T)_SPC'] + get_energy(base_reactants_df, 'boh3')
    else:
        print(f"Messed up {row}")

In [19]:
unlig_df['equalized_gibbs'] = unlig_df.apply(lambda row: equalize_unlig_reference(row), axis=1)
unlig_df['reference_gibbs'] = unlig_df.apply(lambda row: (row['equalized_gibbs'] - unlig_df.loc[unlig_df['structure'] == '00-murxt', 'equalized_gibbs'])*2625.5/4.184, axis=1)

In [20]:
base_diffusion_consideration_dict = {"INT2 // H$_2$O": ["INT2", "INT2 + H$_2$O"],
                       "INT3 // B(OH)$_3$": ["INT3 + B(OH)$_3$", "INT3"],
                       "INT4-PDT": ['INT4', 'PDT'],
                       "Pd-RXT": ["$\mu$-RXT", "RXT",],
                        }

pdt_energy = (get_reactant_energy('naphh') + get_reactant_energy('boh3') - get_reactant_energy('naphboh2') - get_reactant_energy('h2o'))*2625.5/4.184

diffusion_consideration_dict = dict(base_diffusion_consideration_dict)
os.makedirs(f"microkinetics/unlig-pdoh2", exist_ok=True)
current_df = unlig_df.copy()
current_df['label_name'] = current_df.apply(lambda row: nice_structure_names[row['structure']], axis=1)
current_df = pd.concat([current_df, pd.DataFrame([{'structure': 'pdt', 'reference_gibbs': pdt_energy, 'label_name': 'PDT'}])], ignore_index=True)
for diffusion, consideration in diffusion_consideration_dict.items():
    #print(consideration)
    energy = max(get_reference_gibbs_energy(current_df, consideration[0]), get_reference_gibbs_energy(current_df, consideration[1]))+diff_barrier
    current_df = pd.concat([current_df, pd.DataFrame([{'structure': 'diffusion', 'reference_gibbs': energy, 'label_name': diffusion}])], ignore_index=True)  
try:
    template = pd.read_csv(f"microkinetics/unlig-pdoh2/unlig-pdoh2-plot.csv")
    template = pd.merge(template, current_df[['label_name', 'reference_gibbs']], on='label_name', how='left', suffixes=('_old', ""))
    template = template.drop('reference_gibbs_old', axis=1)
    template.sort_values(by=['line', 'point'], ascending=[True, True], inplace=True)
    template.to_csv(f"microkinetics/unlig-pdoh2/unlig-pdoh2-plot.csv", index=False)
except FileNotFoundError:
    template = pd.read_csv(f"templates/rpd_template_unlig-pdoh2.csv")
    template = pd.merge(template, current_df, on='label_name')
    template.to_csv(f"microkinetics/unlig-pdoh2/unlig-pdoh2-plot.csv", index=False)

fig, ax = plt.subplots(figsize=(6,3), dpi=300)
for line in set(template['line'].values):
    curr_line_df = template[template['line'] == line].copy().sort_values(by='point').reset_index(drop=True)
    x_solid, y_solid = interpolate(curr_line_df, line=line)

    for idx, point in curr_line_df.iterrows():
        if point['is_point'] == True:
            ax.scatter(point['point'], point['reference_gibbs'], color=point['point_color'], marker=point['point_marker'], s=10)
        if point['is_TS'] == True:
            # Locate triangle marker index
            idx_0 = np.where(x_solid >= curr_line_df.at[idx-1, 'point'])[0][0]
            idx_1 = np.where(x_solid >= point['point'])[0][0]
            idx_2 = np.where(x_solid >= curr_line_df.at[idx+1, 'point'])[0][0]
            ax.plot(x_solid[idx_0:idx_2], y_solid[idx_0:idx_2], color=curr_line_df['line_color'].values[0], 
                    zorder=-1, linewidth=1, linestyle='--' if point['point_marker'] == 'x' or point['point_marker'] == 'v' else '-')
        
        if point['is_label'] == True:
            label_text = f"{point['label_name']}" f"\n{point['reference_gibbs']:.1f}"

            ax.annotate(label_text, xy=(point['point'], point['reference_gibbs']), 
                        ha='center', color=point['point_color'], va=point['vertical_alignment'], 
                        fontsize=5, xytext=(point['point'], point['reference_gibbs']+point['vertical_offset']))
        
    
            
ax.spines[['right', 'top', 'bottom']].set_visible(False)           
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.tick_params(left = False,bottom=False) 
ax.set_xlim(left=-1)
ax.plot(-1, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False, markersize=2)
ax.set_ylabel("$\Delta G$ / $kcal$ $mol^{-1}$", size=8)
ax.set_ylim(-45, 30)
plt.title(f"Unligated Pd(OH)$_2$", size=8)
plt.savefig(f"microkinetics/unlig-pdoh2/unlig-pdoh2-plot.png", facecolor='white', transparent=False, bbox_inches='tight')
plt.close()

## Supplementary Discussion 6.1: Palladium reference states

### Table S20

In [21]:
def get_energy_group_specific(df, ligand, structure_name):
    return df[(df['group'] == ligand) & (df['structure'] == structure_name)]['qh-G(T)_SPC'].values[0]

def construct_reference_state_table(alternative_structure_names, df, ligand, base_reactants_df):
    # Build energy dict
    energies = {name: get_energy_group_specific(df, ligand, name) for name in alternative_structure_names}
    for _, row in base_reactants_df.iterrows():
        energies[row['structure']] = row['qh-G(T)_SPC']

    d = {}

    d['RXT → RXT'] = energies['01-rxt'] - energies['01-rxt']
    d['½ μA-Ref + RB(OH)$_2$ → RXT'] = energies['01-rxt'] - 0.5*energies['00-lpdoh2'] - energies['naphboh2']
    d['½ μB-Ref + RB(OH)$_2$ → RXT'] = energies['01-rxt'] - 0.5*energies['46-ub-ref'] - energies['naphboh2']
    d['½ OAcμ-Ref + 2 H$_2$O + RB(OH)$_2$ → RXT + 2 AcOH'] = energies['01-rxt'] + 2*energies['acoh'] - 0.5*energies['47-oacu-ref'] - 2*energies['h2o'] - energies['naphboh2'] 
    d['C-Ref + RB(OH)$_2$ → RXT'] = energies['01-rxt'] - energies['48-c-ref'] - energies['naphboh2']
    d['T-Ref + RB(OH)$_2$ → RXT'] = energies['01-rxt'] - energies['49-t-ref'] - energies['naphboh2']
    d['CL-Ref + RB(OH)$_2$ → RXT + L'] = energies['01-rxt'] + energies['84-lig'] - energies['50-cl-ref'] - energies['naphboh2']
    d['TL-Ref + RB(OH)$_2$ → RXT + L'] = energies['01-rxt'] + energies['84-lig'] - energies['51-tl-ref'] - energies['naphboh2']
    d['CS-Ref + RB(OH)$_2$ → RXT + S'] = energies['01-rxt'] + energies['14dioxane'] - energies['52-cs-ref'] - energies['naphboh2']
    d['TS-Ref + RB(OH)$_2$ → RXT + S'] = energies['01-rxt'] + energies['14dioxane'] - energies['53-ts-ref'] - energies['naphboh2']
    d['CW-Ref + RB(OH)$_2$ → RXT + H$_2$O'] = energies['01-rxt'] + energies['h2o'] - energies['54-cw-ref'] - energies['naphboh2']
    d['TW-Ref + RB(OH)$_2$ → RXT + H$_2$O'] = energies['01-rxt'] + energies['h2o'] - energies['55-tw-ref'] - energies['naphboh2']
    d['OH-Ref + RB(OH)$_2$ → RXT + $^-$OH'] = energies['01-rxt'] + energies['oh0dioxane'] - energies['56-oh-ref'] - energies['naphboh2']
    d['KOH-Ref + RB(OH)$_2$ → RXT + KOH'] = energies['01-rxt'] + energies['koh'] - energies['57-koh-ref'] - energies['naphboh2']
    d['KOH-Ref + K$_2$HPO4$^-$ + RB(OH)$_2$ → RXT + [K$_3$PO$_4$--H$_2$O]'] = energies['01-rxt'] + energies['k3po4h2o'] - energies['57-koh-ref'] - energies['k2hpo4'] - energies['naphboh2']
    d['KOH-Ref + K$_2$HPO4$^-$ + RB(OH)$_2$ → RXT + K$_3$PO$_4$ + H$_2$O'] = energies['01-rxt'] + energies['k3po4'] + energies['h2o'] - energies['57-koh-ref'] - energies['k2hpo4'] - energies['naphboh2']
    d['Pd(OAc)$_2$-Ref + 2 H$_2$O + L + RB(OH)$_2$ → RXT + 2 AcOH'] = energies['01-rxt'] + 2*energies['acoh'] - energies['pdoac2'] - 2*energies['h2o'] - energies['84-lig'] - energies['naphboh2']
    d['½ Pd$_2$(OAc)$_4$-Ref + 2 H$_2$O + L + RB(OH)$_2$ → RXT + 2 AcOH'] = energies['01-rxt'] + 2*energies['acoh'] - 0.5*energies['pd2oac4'] - 2*energies['h2o'] - energies['84-lig']  - energies['naphboh2']
    d['⅓ Pd$_3$(OAc)$_6$-Ref + 2 H$_2$O + L + RB(OH)$_2$ → RXT + 2 AcOH'] = energies['01-rxt'] + 2*energies['acoh']  - 1/3*energies['pd3oac6'] - 2*energies['h2o'] - energies['84-lig']  - energies['naphboh2']
    d['¼ Pd$_4$(OAc)$_8$-Ref + 2 H$_2$O + L + RB(OH)$_2$ → RXT + 2 AcOH'] = energies['01-rxt'] + 2*energies['acoh']  - 0.25*energies['pd4oac8'] - 2*energies['h2o'] - energies['84-lig']  - energies['naphboh2']

    
    for key in d.keys():
        d[key] = d[key]*2625.5/4.184
    return d


In [22]:
alternative_structure_names = ['01-rxt', '00-lpdoh2'] + list(nice_alternative_names.keys()) + ['84-lig']
alternative_structure_names.remove('58-c2alt')

In [23]:
alternative_structure_names

['01-rxt',
 '00-lpdoh2',
 '46-ub-ref',
 '47-oacu-ref',
 '48-c-ref',
 '49-t-ref',
 '50-cl-ref',
 '51-tl-ref',
 '52-cs-ref',
 '53-ts-ref',
 '54-cw-ref',
 '55-tw-ref',
 '56-oh-ref',
 '57-koh-ref',
 '70-c2dim',
 '84-lig']

In [24]:
reference_dict = {}
for ligand in ['2g-pcy3', '3d-cyjohnphos', '2j-ptbu3', '3k-tbujohnphos']:
    reference_dict[ligand] = construct_reference_state_table(alternative_structure_names, df, ligand, base_reactants_df)

In [25]:
reference_df = pd.DataFrame(reference_dict)
reference_df.to_csv("supp_table_s20.csv")

### Supplementary Table S21

In [26]:
def get_oh_dioxane_energy(oh_dioxane_df, energy_type, structure):
    return oh_dioxane_df.loc[(oh_dioxane_df['energy_type'] == energy_type) & (oh_dioxane_df['structure'] == structure)]['qh-G(T)'].values[0]

In [30]:
oh_dioxane_df = pd.read_csv("supp_table_s21.csv", skiprows=23)
oh_dioxane_df = oh_dioxane_df.tail(-1)
oh_dioxane_df = oh_dioxane_df.head(-1)
oh_dioxane_df = oh_dioxane_df[['   Structure', 'qh-G(T)', 'im']]
oh_dioxane_df.rename(columns={'   Structure': 'filename'}, inplace=True)
oh_dioxane_df['filename'] = oh_dioxane_df.apply(lambda row: row['filename'].split()[1], axis=1)
oh_dioxane_df['structure'] = oh_dioxane_df.apply(lambda row: row['filename'].split('-')[2], axis=1)
oh_dioxane_df['energy_type'] = oh_dioxane_df.apply(lambda row: row['filename'].split('-')[3], axis=1)

In [31]:
d = {}
for energy_type in oh_dioxane_df['energy_type']:
    d[energy_type] = {}
    reference_energy = 2*get_oh_dioxane_energy(oh_dioxane_df, energy_type, "14dioxane") + get_oh_dioxane_energy(oh_dioxane_df, energy_type, "oh0dioxane")
    for dioxane_num in ["0", "1", "2"]:
        d[energy_type][f'[$^-$OH -- {2-int(dioxane_num)} dioxane] + {dioxane_num}dioxane'] = ((2-int(dioxane_num))*get_oh_dioxane_energy(oh_dioxane_df, energy_type, "14dioxane") + get_oh_dioxane_energy(oh_dioxane_df, energy_type, f"oh{dioxane_num}dioxane") - reference_energy)*2625.5/4.184

In [32]:
oh_dioxane_df_results = pd.DataFrame(d).T
oh_dioxane_df_results

,[$^-$OH -- 2 dioxane] + 0dioxane,[$^-$OH -- 1 dioxane] + 1dioxane,[$^-$OH -- 0 dioxane] + 2dioxane
gassvp,0.0,-43.657723,-59.137128
smdqzvpp,0.0,5.045177,9.081946


### Data for Supplementary Figure S46-S49 and Table S22

In [33]:
def construct_pd3oac6_table(alternative_structure_names, df, ligand_list, base_reactants_df):
    
    pd3oac6_dict = {}
    for ligand in ligand_list:
        pd3oac6_dict[ligand] = {}
        energies = {name: get_energy_group_specific(df, ligand, name) for name in alternative_structure_names}
        for _, row in base_reactants_df.iterrows():
            energies[row['structure']] = row['qh-G(T)_SPC']
        # Raw energy
        pd3oac6_dict[ligand]['raw'] = 1/3*energies['pd3oac6'] + 2*energies['h2o'] + energies['84-lig'] - 2*energies['acoh'] 
        # Further reference equalization
        pd3oac6_dict[ligand]['ref'] = pd3oac6_dict[ligand]['raw'] + 2*energies['naphboh2'] + 2*energies['h2o'] + energies['84-lig']

    return pd3oac6_dict

In [34]:
ligand_list = ['2g-pcy3', '3d-cyjohnphos', '2j-ptbu3', '3k-tbujohnphos']
pd3oac6_dict = construct_pd3oac6_table(alternative_structure_names, df, ligand_list, base_reactants_df)

In [35]:
pd3oac6_dict

{'2g-pcy3': {'raw': -1324.9703779999998, 'ref': -3645.626157},
 '3d-cyjohnphos': {'raw': -1552.124654, 'ref': -4099.934709000001},
 '2j-ptbu3': {'raw': -1093.0614439999997, 'ref': -3181.808289},
 '3k-tbujohnphos': {'raw': -1397.5243369999998, 'ref': -3790.7340750000003}}

In [36]:
pd3oac6_main_df = main_df[main_df['group'].isin(ligand_list)].copy()

In [37]:
pd3oac6_main_df['equalized_gibbs'] = pd3oac6_main_df.apply(lambda row: equalize_reference(row, row['group']), axis=1)
pd3oac6_main_df['reference_gibbs'] = pd3oac6_main_df.apply(lambda row: (row['equalized_gibbs'] - pd3oac6_dict[row['group']]['ref'])*2625.5/4.184, axis=1)

In [38]:
diff_barrier = 3.4663630747609946
type_list = ['pd3oac6']

for type_name in type_list:
    int_list, ts_list, rxt_pdt_list, diffusion_ts_list, xform_dict = get_main_microkinetics_transformations(type_name='main') # Follows main reaction pathway

    for ligand in ligand_list:
        os.makedirs(f"microkinetics/{type_name}-{ligand}", exist_ok=True)
        print(f"{type_name}-{ligand}")
        current_df = pd3oac6_main_df[pd3oac6_main_df['group'] == ligand][['structure', 'qh-G(T)_SPC']].copy()
        current_df = pd.concat([current_df, pd.DataFrame([{'structure': '99-l2pd2oh4', 'qh-G(T)_SPC': current_df[current_df['structure'] == '00-lpdoh2']['qh-G(T)_SPC'].values[0]}])], ignore_index=True)
        current_df.loc[current_df['structure'] == '00-lpdoh2', 'qh-G(T)_SPC'] = pd3oac6_dict[ligand]['raw'] # Spoofing 00-lpdoh2 energies with pd3oac6-ref data
        int_df = current_df[(~current_df['structure'].str.contains('ts')) & (current_df['structure'].isin(int_list))].copy()
        
        for rxt in rxt_pdt_list:
            int_df = pd.concat([int_df, pd.DataFrame([{'structure': rxt, 'qh-G(T)_SPC': get_reactant_energy(rxt)}])], ignore_index=True)
        

        int_df.to_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-intermediates.csv", index=False)

        xform_df = pd.DataFrame(columns=['I1', 'I2', 'P1', 'P2', 'Energy', 'Backwards', 'Flag', 'Ref_L', 'Ref_R'])
        xform_df["Backwards"] = xform_df["Backwards"].astype(bool)

        for ts, xform in xform_dict.items():
            energy_l, energy_r = get_left_right_barrier_reference_energies(int_df, xform)

            if ts in diffusion_ts_list:
                ts_energy = max(energy_l, energy_r)
            else:
                ts_energy = get_energy(current_df, ts)

            xform_df = pd.concat([xform_df, pd.DataFrame([{'I1': xform[0], 'I2': xform[1], 'P1': xform[2], 'P2': xform[3], 
                                                        'Energy': ts_energy, 'Backwards': xform[4], 'Flag': ts, 
                                                        'Ref_L': (ts_energy-energy_l)*2625.5/4.184, 
                                                        'Ref_R': (ts_energy-energy_r)*2625.5/4.184,}],
                                                        )])

        xform_df.to_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-transformations.csv", index=False)    


pd3oac6-2g-pcy3
pd3oac6-3d-cyjohnphos
pd3oac6-2j-ptbu3
pd3oac6-3k-tbujohnphos


In [39]:
diffusion_consideration_dict = {"C2 // H$_2$O": ["C2", "C2 + H$_2$O"],
                       "C3 // B(OH)$_3$": ["C3 + B(OH)$_3$", "C3"],
                       "C4-PDT": ['C4', 'PDT'],
                       "T2 // H$_2$O": ["T2", "T2 + H$_2$O"],
                       "T3 // B(OH)$_3$": ["T3 + B(OH)$_3$", "T3"],
                       'T4-PDT': ["T4", "PDT"],
                       "Pd-RXT": ["Pd$_3$(OAc)$_6$ + L", "RXT",],
                        }

pdt_energy = (get_reactant_energy('naphh') + get_reactant_energy('boh3') - get_reactant_energy('naphboh2') - get_reactant_energy('h2o'))*2625.5/4.184

for type_name in type_list:

    for ligand in ligand_list:
        print(f"{type_name}-{ligand}")
        os.makedirs(f"microkinetics/{type_name}-{ligand}", exist_ok=True)
        current_df = pd3oac6_main_df[pd3oac6_main_df['group'] == ligand][['structure', 'reference_gibbs']].copy()
        current_df['label_name'] = current_df.apply(lambda row: nice_structure_names[row['structure']], axis=1)
        current_df.loc[current_df['structure'] == '00-lpdoh2', 'reference_gibbs'] = 0 # Spoof reference number
        current_df = pd.concat([current_df, pd.DataFrame([{'structure': 'pd3oac6', 'reference_gibbs': 0.0, 'label_name': 'Pd$_3$(OAc)$_6$ + L'}])], ignore_index=True)
        current_df = pd.concat([current_df, pd.DataFrame([{'structure': 'pdt', 'reference_gibbs': pdt_energy, 'label_name': 'PDT'}])], ignore_index=True)
        for diffusion, consideration in diffusion_consideration_dict.items():
            #print(consideration)
            energy = max(get_reference_gibbs_energy(current_df, consideration[0]), get_reference_gibbs_energy(current_df, consideration[1]))+diff_barrier
            current_df = pd.concat([current_df, pd.DataFrame([{'structure': 'diffusion', 'reference_gibbs': energy, 'label_name': diffusion}])], ignore_index=True)
            
        try:
            template = pd.read_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot.csv")
            template = pd.merge(template, current_df[['label_name', 'reference_gibbs']], on='label_name', how='left', suffixes=('_old', ""))
            template = template.drop('reference_gibbs_old', axis=1)
            template.to_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot.csv", index=False)
        except FileNotFoundError:
            template = pd.read_csv(f"templates/rpd_template_main.csv")
            template = pd.merge(template, current_df, on='label_name')
            template.to_csv(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot.csv", index=False)

        fig, ax = plt.subplots(figsize=(6,3), dpi=300)
        for line in set(template['line'].values):
            curr_line_df = template[template['line'] == line].copy().sort_values(by='point').reset_index(drop=True)
            x_solid, y_solid = interpolate(curr_line_df, line=line)

            for idx, point in curr_line_df.iterrows():
                if point['is_point'] == True:
                    ax.scatter(point['point'], point['reference_gibbs'], color=point['point_color'], marker=point['point_marker'], s=10)
                if point['is_TS'] == True:
                    # Locate triangle marker index
                    idx_0 = np.where(x_solid >= curr_line_df.at[idx-1, 'point'])[0][0]
                    idx_1 = np.where(x_solid >= point['point'])[0][0]
                    idx_2 = np.where(x_solid >= curr_line_df.at[idx+1, 'point'])[0][0]
                    ax.plot(x_solid[idx_0:idx_2], y_solid[idx_0:idx_2], color=curr_line_df['line_color'].values[0], 
                            zorder=-1, linewidth=1, linestyle='--' if point['point_marker'] == 'x' else '-')
                
                if point['is_label'] == True:
                    label_text = f"{point['label_name']}" f"\n{point['reference_gibbs']:.1f}"

                    ax.annotate(label_text, xy=(point['point'], point['reference_gibbs']), 
                                ha='center', color=point['point_color'], va=point['vertical_alignment'], 
                                fontsize=5, xytext=(point['point'], point['reference_gibbs']+point['vertical_offset']))
                
            
                    
        ax.spines[['right', 'top', 'bottom']].set_visible(False)           
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        plt.tick_params(left = False,bottom=False) 
        ax.set_xlim(left=-1.5)
        ax.plot(-1.5, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False, markersize=2)
        ax.set_ylabel("$\Delta G$ / $kcal$ $mol^{-1}$", size=8)
        ax.set_ylim(-45, 30)
        plt.title(f"L = {proper_names_dict[ligand]}", size=8)
        plt.savefig(f"microkinetics/{type_name}-{ligand}/{type_name}-{ligand}-plot.png", facecolor='white', transparent=False, bbox_inches='tight')
        plt.close()

pd3oac6-2g-pcy3
pd3oac6-3d-cyjohnphos
pd3oac6-2j-ptbu3
pd3oac6-3k-tbujohnphos


## Supplementary Discussion 6.2

In [40]:
main_df[((main_df['structure'] == '58-c2alt') | (main_df['structure'] == '05-c2')) & (main_df['group'].isin(ligand_list))]

,filename,qh-G(T)_SPC,im,group,structure,equalized_gibbs,reference_gibbs
135,2g-pcy3-05-c2,-1886.169783,C1,2g-pcy3,05-c2,-3645.675186,-25.122345
175,2g-pcy3-58-c2alt,-1886.163040,C1,2g-pcy3,58-c2alt,-3645.668443,-20.891048
250,2j-ptbu3-05-c2,-1654.248268,C1,2j-ptbu3,05-c2,-3181.844737,-25.382762
283,2j-ptbu3-58-c2alt,-1654.241620,C1,2j-ptbu3,58-c2alt,-3181.838089,-21.211078
425,3d-cyjohnphos-05-c2,-2113.320367,C1,3d-cyjohnphos,05-c2,-4099.980046,-26.712768
471,3d-cyjohnphos-58-c2alt,-2113.319029,C1,3d-cyjohnphos,58-c2alt,-4099.978708,-25.873160
653,3k-tbujohnphos-05-c2,-1958.710004,C1,3k-tbujohnphos,05-c2,-3790.769366,-24.496404
693,3k-tbujohnphos-58-c2alt,-1958.705733,C1,3k-tbujohnphos,58-c2alt,-3790.765095,-21.816311


## Supplementary Discussion 6.3

In [41]:
sd_63_structures = ['05-c2', '06-c2-h2o', '07-ts-c2-c3', '10-ts-c3-c4', '22-c5', '23-ts-c5-c6', '15-t2', '16-t2-h2o', '17-ts-t2-t3', '20-ts-t3-t4', '24-t5', '25-ts-t5-t6']

sd_63_df = main_df[(main_df['structure'].isin(sd_63_structures)) & (main_df['group'].isin(structure_dict['altpdb']))].copy()
sd_63_df = sd_63_df.pivot(index='group', columns='structure', values='reference_gibbs')
sd_63_df = sd_63_df[sd_63_structures]

sd_63_df.to_csv('supp_table_s24.csv')

In [42]:
nice_main_names.keys()

dict_keys(['00-lpdoh2', '01-rxt', '02-ts-rxt-c1', '03-c1', '04-ts-c1-c2', '05-c2', '06-c2-h2o', '07-ts-c2-c3', '08-c3-boh3', '09-c3', '10-ts-c3-c4', '11-c4', '12-ts-rxt-t1', '13-t1', '14-ts-t1-t2', '15-t2', '16-t2-h2o', '18-t3-boh3', '17-ts-t2-t3', '19-t3', '20-ts-t3-t4', '21-t4'])

## Supplementary Table S27

In [43]:
tunnelling_ligands = ['3o-tbubrettphos', '3p-me4tbuxphos']
tunnelling_structures = np.array([['15-t2', '25-ts-t5-t6', '26-t6']])
tunnelling_df = main_df[(main_df['group'].isin(tunnelling_ligands)) & (main_df['structure'].isin(tunnelling_structures.flatten()))].copy()

In [44]:
for group in tunnelling_ligands:
    tmp = tunnelling_df[tunnelling_df['group'] == group].copy()
    for structures in tunnelling_structures:
        print(group, structures[1])
        im = tmp[tmp['structure'] == structures[1]]['im'].values[0]
        E0_reac = tmp[tmp['structure'] == structures[0]]['qh-G(T)_SPC'].values[0] + get_reactant_energy('h2o')
        E0_TS = tmp[tmp['structure'] == structures[1]]['qh-G(T)_SPC'].values[0]
        E0_prod = tmp[tmp['structure'] == structures[2]]['qh-G(T)_SPC'].values[0] 
        T = 333.15
        %run tunnelling.py {im} {E0_reac} {E0_TS} {E0_prod} {T}

3o-tbubrettphos 25-ts-t5-t6
Wigner Kappa is 1.4653688771073825
Eckart Kappa is 1.6756443790237958
Uncorrected barrier is 31.329042303818643
Wigner Corrected barrier is 31.07607300601345
Eckart Corrected barrier is 30.9872998274862
3p-me4tbuxphos 25-ts-t5-t6
Wigner Kappa is 1.5992426599839689
Eckart Kappa is 1.9773945747460904
Uncorrected barrier is 31.75888635284601
Wigner Corrected barrier is 31.448039604158176
Eckart Corrected barrier is 31.307522119604055


<Figure size 640x480 with 0 Axes>